In [1]:
import torch
import numpy as np
import pandas as pd
from lstm_aux import LSTMModel
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [2]:
n_features = 6
sequence_length = 20

In [3]:
# Create an instance of the LSTM model
model = LSTMModel(n_features, hidden_size=64)

# Load the saved weights
model.load_state_dict(torch.load('lstm_model_0.485.pt'))
model.eval()  # Set the model to evaluation mode

LSTMModel(
  (lstm1): LSTM(6, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [4]:
input_sequence =   [['normal_trimmed_39.mp4', 1, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 2, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 3, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 4, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 5, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 6, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 7, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 8, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 9, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 10, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 11, 0.0, -1.0, 0, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 12, 0.2613290720593492, 0.0, 1, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 13, 0.2621389838245704, 0.0, 1, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 14, 0.2646246850364757, 0.0, 1, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 15, 0.2616935144854627, 0.0, 1, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 16, 0.2574731656160717, 0.0, 1, 0, 0, 0],
                    ['normal_trimmed_39.mp4', 17, 1.788942931909484, 2.5, 1, 1, 0, 0],
                    ['normal_trimmed_39.mp4', 18, 1.8930430067860773, 2.5, 1, 1, 0, 0],
                    ['normal_trimmed_39.mp4', 19, 1.935407728971768, 2.5, 1, 1, 0, 0],
                    ['normal_trimmed_39.mp4', 20, 1.794614067543686, 2.5, 1, 1, 0, 0]]

input_sequence = np.array(input_sequence)

In [5]:
optimal_threshold = 0.485

temp_vid_id = ''
input_data = input_sequence[:, 2:].astype(np.float32)
input_data_scaled = scaler.fit_transform(input_data)
input_data = torch.tensor(input_data_scaled, dtype=torch.float32) # Convert numpy array to tensor
vid_id, frame_num = input_sequence[sequence_length-1, 0], input_sequence[sequence_length-1, 1]  # ID and frame of last row in the sequence

if vid_id == temp_vid_id:
    RBP = 1  # Ignore succeedig sequences from the same video if robbery prediction was already made

# Make predictions
with torch.no_grad():
    output = model(input_data.unsqueeze(0))  # Add batch dimension
    RBP = (output).squeeze().cpu().numpy()

if RBP >= optimal_threshold:
    temp_vid_id = vid_id
    print('WARNING! High Robbery Potential!')
else:
    print(f'LOW Robbery Potential ({RBP:.3f})')

LOW Robbery Potential (0.471)
